In [1]:
from sqlalchemy import create_engine, text, inspect, func
import sqlite3

# ORM imports
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy import Column, Integer, String, Float, Boolean
from sqlalchemy.orm import Session
from sqlalchemy.ext.declarative import declarative_base

# API
import requests
import json

# Data Science and Visualization
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from scipy import stats
import datetime as dt

In [2]:
#CSV import for crashes table
filepath = "Datasets/SGO-2021-01_Incident_Reports_ADS.csv"
df = pd.read_csv(filepath)
df.head()

,Report ID,Report Version,Reporting Entity,Report Type,Report Month,Report Year,Report Submission Date,VIN,VIN - Unknown,Serial Number,...,Investigating Officer Name,Inv. Officer Name - Unknown,Investigating Officer Phone,Inv. Officer Phone - Unknown,Investigating Officer Email,Inv. Officer Email - Unknown,Within ODD?,Within ODD? - CBI,Narrative,Narrative - CBI?
0,30270-7157,3,Waymo LLC,Update,NaN,NaN,AUG-2024,SADHW2S13M1,,NaN,...,NaN,,NaN,,NaN,,Yes,,Waymo is submitting a second update to the rep...,NaN
1,30270-8406,1,Waymo LLC,Monthly,7.0,2024.0,AUG-2024,SADHW2S1XR1,,NaN,...,NaN,,NaN,,NaN,,Yes,,"On July [XXX], 2024 at 4:58 AM PT a Waymo Auto...",NaN
2,30270-8405,1,Waymo LLC,Monthly,7.0,2024.0,AUG-2024,SADHW2S17R1,,NaN,...,NaN,,NaN,,NaN,,Yes,,"On July [XXX], 2024 at 11:29 AM MT a Waymo Aut...",NaN
3,30270-8404,1,Waymo LLC,Monthly,7.0,2024.0,AUG-2024,SADHW2S1XR1,,NaN,...,NaN,,NaN,,NaN,,Yes,,"On July [XXX], 2024 at 4:16 PM PT a Waymo Auto...",NaN
4,30270-8403,1,Waymo LLC,Monthly,7.0,2024.0,AUG-2024,SADHW2S19R1,,NaN,...,NaN,,NaN,,NaN,,Yes,,"On July [XXX], 2024 at 11:06 AM CST a Waymo Au...",NaN


In [3]:
df.columns

Index(['Report ID', 'Report Version', 'Reporting Entity', 'Report Type',
       'Report Month', 'Report Year', 'Report Submission Date', 'VIN',
       'VIN - Unknown', 'Serial Number',
       ...
       'Investigating Officer Name', 'Inv. Officer Name - Unknown',
       'Investigating Officer Phone', 'Inv. Officer Phone - Unknown',
       'Investigating Officer Email', 'Inv. Officer Email - Unknown',
       'Within ODD?', 'Within ODD? - CBI', 'Narrative', 'Narrative - CBI?'],
      dtype='object', length=137)

In [ ]:
#First Set of Columns to remove
# columns_to_remove = ['Report ID', 'Report Version', 'Reporting Entity', 'Report Submission Date', 'VIN', 'VIN – Unknown', 'Serial Number',\
#                     'Make', 'Model', 'Model – Unknown', 'Model Year',	'Model Year – Unknown',	'Same Vehicle ID', 'Mileage', 'Mileage – Unknown',\
#                     'ADAS/ADS System Version', 'ADAS/ADS System Version – Unk', 'ADAS/ADS System Version CBI',  'ADAS/ADS Hardware Version',\
#                     'ADAS/ADS Hardware Version – Unk', 'ADAS/ADS Hardware Version CBI', 'ADAS/ADS Software Version', 'ADAS/ADS Software Version – Unk',\
#                     'ADAS/ADS Software Version CBI', 'Other Reporting Entities?', 'Other Reporting Entities? – Unk', 'Other Reporting Entities? – NA',\
#                     'Federal Regulatory Exemption?', 'Other Federal Reg. Exemption', 'Federal Reg. Exemption – Unk', 'Federal Reg. Exemption – No',\
#                     'State or Local Permit?', 'State or Local Permit', 'ADS Equipped?, Automation System Engaged?',	'Operating Entity',\
#                     'Operating Entity – Unknown', 'Source - Complaint/Claim', 'Source – Telematics', 'Source - Law Enforcement', 'Source - Field Report',\
#                     'Source – Testing', 'Source – Media', 'Source – Other', 'Source - Other Text', 'Notice Received Date', 'Same Incident ID', 'Address',\
#                     'Address – Unknown', 'Crash With', 'Property Damage?', 'CP Pre-Crash Movement', 'CP Any Air Bags Deployed?', 'CP Was Vehicle Towed?',\
#                     'CP Contact Area - Rear Left', 'CP Contact Area – Left', 'CP Contact Area - Front Left', 'CP Contact Area – Rear', 'CP Contact Area – Top',\
#                     'CP Contact Area – Front', 'CP Contact Area - Rear Right', 'CP Contact Area – Right', 'CP Contact Area - Front Right',\
#                     'CP Contact Area – Bottom', 'CP Contact Area – Unknown', 'SV Pre-Crash Movement',	'SV Any Air Bags Deployed?', 'SV Was Vehicle Towed?',\
#                     'SV Contact Area - Rear Left', 'SV Contact Area – Left', 'SV Contact Area - Front Left', 'SV Contact Area – Rear', 'SV Contact Area – Top',\
#                     'SV Contact Area – Front', 'SV Contact Area - Rear Right', 'SV Contact Area – Right', 'SV Contact Area - Front Right',\
#                     'SV Contact Area – Bottom', 'SV Contact Area – Unknown', 'Data Availability – EDR', 'Data Availability - Police Rpt',\
#                     'Data Availability – Telematics', 'Data Availability – Complaints', 'Data Availability – Video', 'Data Availability – Other',\
#                     'Data Availability - No Data', 'Data Availability – Unknown', 'Law Enforcement Investigating?', 'Investigating Agency',\
#                     'Investigating Agency – Unknown', 'Rep Ent Or Mfr Investigating?', 'Investigating Officer Name', 'Inv. Officer Name – Unknown',\
#                     'Investigating Officer Phone', 'Inv. Officer Phone – Unknown', 'Investigating Officer Email', 'Inv. Officer Email – Unknown', 'Within ODD?',\
#                     'Within ODD? – CBI', 'Narrative', 'Narrative - CBI?']

# df2 = df.drop(columns_to_remove, axis=1)



In [4]:
cols_to_drop = ['Report ID', 'Report Version', 'Report Month', 'Report Year','Reporting Entity','Report Submission Date',\
                'VIN', 'Make', 'Model','Model - Unknown', 'Model Year', 'Model Year - Unknown',\
                'Same Vehicle ID', 'Mileage','VIN - Unknown', 'Serial Number',\
               'Make', 'Model','Model - Unknown', 'Model Year', 'Model Year - Unknown','Same Vehicle ID', 'Mileage',\
               'Report Type', 'Mileage - Unknown', 'Driver / Operator Type', 'ADAS/ADS System Version','ADAS/ADS System Version - Unk',\
                'ADAS/ADS System Version CBI','ADAS/ADS Hardware Version', 'ADAS/ADS Hardware Version - Unk','Investigating Officer Name',\
                'Inv. Officer Name - Unknown', 'Investigating Officer Phone', 'Inv. Officer Phone - Unknown','Investigating Officer Email',\
                'Inv. Officer Email - Unknown','Within ODD?', 'Within ODD? - CBI', 'Narrative', 'Narrative - CBI?','ADAS/ADS Hardware Version CBI',\
               'ADAS/ADS Software Version', 'ADAS/ADS Software Version - Unk',\
               'ADAS/ADS Software Version CBI', 'Other Reporting Entities?',\
               'Other Reporting Entities? - Unk', 'Other Reporting Entities? - NA',\
               'Federal Regulatory Exemption?',\
               'Data Availability - Telematics', 'Data Availability - Complaints',\
               'Data Availability - Video', 'Data Availability - Other',\
               'Data Availability - No Data', 'Data Availability - Unknown',\
               'Law Enforcement Investigating?', 'Investigating Agency',\
               'Investigating Agency - Unknown', 'Rep Ent Or Mfr Investigating?','Other Federal Reg. Exemption',\
               'Federal Reg. Exemption - Unk', 'Federal Reg. Exemption - No',\
               'State or Local Permit?', 'State or Local Permit', 'ADS Equipped?',\
               'Automation System Engaged?', 'Source - Complaint/Claim',\
               'Source - Telematics', 'Source - Law Enforcement',\
               'Source - Field Report', 'Source - Testing', 'Source - Media',\
               'Source - Other', 'Source - Other Text','Incident Date - Unknown', 'Notice Received Date',\
               'Same Incident ID','Address', 'Address - Unknown','Property Damage?',\
               'CP Pre-Crash Movement', 'CP Any Air Bags Deployed?','CP Was Vehicle Towed?', 'CP Contact Area - Rear Left',\
               'CP Contact Area - Left', 'CP Contact Area - Front Left','CP Contact Area - Rear', 'CP Contact Area - Top',\
               'CP Contact Area - Front', 'CP Contact Area - Rear Right','CP Contact Area - Right', 'CP Contact Area - Front Right',\
               'CP Contact Area - Bottom', 'CP Contact Area - Unknown','SV Pre-Crash Movement', 'SV Any Air Bags Deployed?',\
               'SV Was Vehicle Towed?','SV Precrash Speed (MPH)', 'SV Pre-crash Speed - Unknown','SV Contact Area - Rear Left',\
                'SV Contact Area - Left','SV Contact Area - Front Left', 'SV Contact Area - Rear','SV Contact Area - Top',\
                'SV Contact Area - Front','SV Contact Area - Rear Right', 'SV Contact Area - Right','SV Contact Area - Front Right',\
                'SV Contact Area - Bottom','SV Contact Area - Unknown', 'Data Availability - EDR','Data Availability - Police Rpt',\
               'Operating Entity','Operating Entity - Unknown','Crash With', 'Weather - Unknown','Incident Time - Unknown', 'Latitude',\
                'Latitude - Unknown','Longitude', 'Longitude - Unknown','City - Unknown','Zip Code','Zip Code - Unknown',\
                'Posted Speed Limit - Unknown']
df2 = df.drop(cols_to_drop, axis=1)
df2.head()

,Incident Date,Incident Time (24:00),City,State,Roadway Type,Roadway Surface,Roadway Description,Posted Speed Limit (MPH),Lighting,Weather - Clear,Weather - Snow,Weather - Cloudy,Weather - Fog/Smoke,Weather - Rain,Weather - Severe Wind,Weather - Other,Weather - Other Text,Highest Injury Severity Alleged,SV Were All Passengers Belted?
0,JAN-2024,12:04,Los Angeles,CA,Street,Dry,No Unusual Conditions,25.0,Daylight,Y,,,,,,,NaN,Unknown,"No, see Narrative"
1,JUL-2024,04:58,San Francisco,CA,Street,Dry,No Unusual Conditions,25.0,Dark - Lighted,Y,,,,,,,NaN,No Injuries Reported,No Passengers in Vehicle
2,JUL-2024,11:29,Phoenix,AZ,Parking Lot,Dry,No Unusual Conditions,5.0,Daylight,Y,,,,,,,NaN,No Injuries Reported,Yes
3,JUL-2024,18:16,San Francisco,CA,Street,Dry,No Unusual Conditions,25.0,Daylight,Y,,,,,,,NaN,No Injuries Reported,No Passengers in Vehicle
4,JUL-2024,11:06,Austin,TX,Street,Dry,No Unusual Conditions,40.0,Daylight,,,Y,,,,,NaN,No Injuries Reported,Yes


In [5]:
# Refreshed look at columns
df2.columns

Index(['Incident Date', 'Incident Time (24:00)', 'City', 'State',
       'Roadway Type', 'Roadway Surface', 'Roadway Description',
       'Posted Speed Limit (MPH)', 'Lighting', 'Weather - Clear',
       'Weather - Snow', 'Weather - Cloudy', 'Weather - Fog/Smoke',
       'Weather - Rain', 'Weather - Severe Wind', 'Weather - Other',
       'Weather - Other Text', 'Highest Injury Severity Alleged',
       'SV Were All Passengers Belted?'],
      dtype='object')

In [6]:
cols = ['Incident Date', 'Incident Time (24:00)','City',  'State','Roadway Type', 'Roadway Surface',\
        'Roadway Description', 'Posted Speed Limit (MPH)', 'Lighting', 'Weather - Clear','Weather - Snow', 'Weather - Cloudy', 'Weather - Fog/Smoke',\
        'Weather - Rain', 'Weather - Severe Wind', 'Weather - Other', 'Weather - Other Text','Highest Injury Severity Alleged',\
        'SV Were All Passengers Belted?']

for col in cols:
  print(col)
  print(df2[col].nunique())
  print(df2[col].value_counts())
  print()

Incident Date
38
Incident Date
AUG-2023    100
SEP-2023     57
OCT-2023     56
MAR-2023     55
MAY-2024     55
JUL-2023     55
JUN-2024     50
AUG-2022     48
JUL-2024     48
JUN-2023     46
MAY-2023     43
APR-2023     42
DEC-2022     41
MAR-2022     37
APR-2022     37
JUN-2022     36
NOV-2021     34
JUL-2022     34
FEB-2023     33
FEB-2024     33
MAR-2024     33
MAY-2022     33
OCT-2021     32
SEP-2022     30
AUG-2021     29
JAN-2024     27
JAN-2022     26
APR-2024     25
SEP-2021     25
JUL-2021     24
DEC-2023     24
FEB-2022     21
DEC-2021     19
NOV-2023     18
NOV-2022     16
OCT-2022     15
JAN-2023     12
AUG-2024      9
Name: count, dtype: int64

Incident Time (24:00)
564
Incident Time (24:00)
03:15    11
00:39    10
21:29    10
22:22    10
08:50     9
         ..
12:03     1
8:01      1
08:01     1
07:38     1
09:50     1
Name: count, Length: 564, dtype: int64

City
88
City
San Francisco       743
Phoenix             101
Tempe                80
Austin               75
Los A

In [8]:
df2['Incident Date'].value_counts()

Incident Date
AUG-2023    100
SEP-2023     57
OCT-2023     56
MAR-2023     55
MAY-2024     55
JUL-2023     55
JUN-2024     50
AUG-2022     48
JUL-2024     48
JUN-2023     46
MAY-2023     43
APR-2023     42
DEC-2022     41
MAR-2022     37
APR-2022     37
JUN-2022     36
NOV-2021     34
JUL-2022     34
FEB-2023     33
FEB-2024     33
MAR-2024     33
MAY-2022     33
OCT-2021     32
SEP-2022     30
AUG-2021     29
JAN-2024     27
JAN-2022     26
APR-2024     25
SEP-2021     25
JUL-2021     24
DEC-2023     24
FEB-2022     21
DEC-2021     19
NOV-2023     18
NOV-2022     16
OCT-2022     15
JAN-2023     12
AUG-2024      9
Name: count, dtype: int64

In [9]:
df2['Incident Time (24:00)'].value_counts()

Incident Time (24:00)
03:15    11
00:39    10
21:29    10
22:22    10
08:50     9
         ..
12:03     1
8:01      1
08:01     1
07:38     1
09:50     1
Name: count, Length: 564, dtype: int64

In [10]:
df2['City'].value_counts()

City
San Francisco       743
Phoenix             101
Tempe                80
Austin               75
Los Angeles          36
                   ... 
Foster City           1
Port Saint Lucie      1
Shenandoah            1
Rice                  1
Sunol                 1
Name: count, Length: 88, dtype: int64

In [11]:
df2['State'].value_counts()

State
CA     852
AZ     240
TX     130
FL      42
NV      35
MI      11
DC       8
NM       6
MN       6
CO       6
RI       5
IN       4
PA       3
OH       3
WA       3
WY       3
GA       1
VA       1
Name: count, dtype: int64

In [12]:
df2['Roadway Type'].value_counts()

Roadway Type
Intersection         592
Street               576
Highway / Freeway    107
Parking Lot           73
Traffic Circle         6
Unknown                5
Name: count, dtype: int64

In [13]:
df2['Roadway Surface'].value_counts()

Roadway Surface
Dry                   1284
Wet                     64
Unknown                  9
Snow / Slush / Ice       2
Name: count, dtype: int64

In [14]:
df2['Roadway Description'].value_counts()

Roadway Description
No Unusual Conditions          1285
Other, see Narrative             37
Work Zone                        19
Traffic Incident                  9
Unknown                           8
Missing / Degraded Markings       1
Name: count, dtype: int64

In [15]:
df2['Posted Speed Limit (MPH)'].value_counts()

Posted Speed Limit (MPH)
25.0    752
30.0    147
35.0    110
45.0     58
15.0     54
65.0     53
20.0     45
40.0     40
75.0     30
5.0      13
10.0      5
55.0      5
0.0       4
70.0      4
60.0      4
50.0      3
9.0       3
1.0       1
Name: count, dtype: int64

In [16]:
df2['Lighting'].value_counts()

Lighting
Daylight                   791
Dark - Lighted             488
Dawn / Dusk                 46
Dark - Not Lighted          26
Unknown                      5
Dark - Unknown Lighting      3
Name: count, dtype: int64

In [17]:
df2['Weather - Clear'].value_counts()

Weather - Clear
Y    1121
      241
Name: count, dtype: int64

In [18]:
df2['Weather - Snow'].value_counts()

Weather - Snow
     1361
Y       1
Name: count, dtype: int64

In [19]:
df2['Weather - Cloudy'].value_counts()

Weather - Cloudy
     1189
Y     173
Name: count, dtype: int64

In [20]:
df2['Weather - Fog/Smoke'].value_counts()

Weather - Fog/Smoke
     1362
Name: count, dtype: int64

In [21]:
df2['Weather - Rain'].value_counts()

Weather - Rain
     1317
Y      45
Name: count, dtype: int64

In [22]:
df2['Weather - Severe Wind'].value_counts()

Weather - Severe Wind
     1362
Name: count, dtype: int64

In [23]:
df2['Weather - Other'].value_counts()

Weather - Other
     1357
Y       5
Name: count, dtype: int64

In [24]:
df2['Weather - Other Text'].value_counts()

Weather - Other Text
Thundering    5
Name: count, dtype: int64

In [25]:
df2['Highest Injury Severity Alleged'].value_counts()

Highest Injury Severity Alleged
No Injuries Reported    1092
Minor                    133
Unknown                   74
Moderate                  37
Serious                   21
Fatality                   2
Name: count, dtype: int64

In [26]:
df2['SV Were All Passengers Belted?'].value_counts()

SV Were All Passengers Belted?
Yes                         876
No Passengers in Vehicle    442
No, see Narrative            30
Unknown                      11
Name: count, dtype: int64